In [ ]:
import torch as tr
import torchvision as trv
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import os
from tempfile import TemporaryDirectory
from imutils import paths
from torch.utils.data.dataset import random_split
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Start

In [ ]:
# три класса, подготовленные из объединенных данных с частью набора
# Caltech Camera Traps (CCT) (Caltech Camera Traps (CCT)) https://lila.science/datasets/caltech-camera-traps
anim = np.array((list(paths.list_images('/data/animals'))))
broken = np.array((list(paths.list_images('/data/broken'))))
empty = np.array((list(paths.list_images('/data/empty'))))

In [ ]:
len(anim),len(broken),len(empty)

(2174, 617, 950)

In [ ]:
# класс для дальнейшей загрузки данных и представления изображений в виде интенсивности
class Data(tr.utils.data.Dataset):
    def __init__(self, dirs, label, transform):
        self.main_dir = dirs
        self.transform = transform
        self.classes = label

    def __len__(self):
        return len(self.main_dir)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir[idx])
        image = Image.open(img_loc).convert("RGB")
        w, h = image.size
        image = image.crop((0, 30, w, h-100)) # обрезка метки на фото
        tensor_image = self.transform(image)
        return tensor_image,self.classes[idx]


In [ ]:
y2 = np.full([len(anim)],2) # метки класса 'animals1
y1 = np.ones(len(empty)) # метки для пустых изображений
y0 = np.zeros(len(broken)) # для поврежденных

X = np.concatenate([anim,empty,broken])
Y = np.concatenate([y2,y1,y0]).astype(int)


data_transforms = trv.transforms.Compose([
        trv.transforms.Resize(256), # сжатие изображения
        trv.transforms.CenterCrop(256), # создание одинаковой формы для всех изображений
        trv.transforms.ToTensor(),
        trv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

dataset = Data(X, Y,data_transforms)

# Split data

In [ ]:
num_train = int(len(dataset) * 0.7)
num_valid = int((len(dataset)-(num_train))*0.3)
split_train, split_test = \
    random_split(dataset, [num_train, len(dataset) - num_train])
split_test_,split_valid =random_split(split_test, [int(len(dataset)-num_train)-num_valid,num_valid])

data_train = tr.utils.data.DataLoader(split_train, batch_size=20, shuffle=True)
data_test = tr.utils.data.DataLoader(split_test_, batch_size=16, shuffle=True)
data_val = tr.utils.data.DataLoader(split_valid, batch_size=16, shuffle=False)

## Next

In [ ]:
import time
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()
    device = tr.device('cuda') if tr.cuda.is_available() else tr.device('cpu')

    total_acc, total_count = 0, 0
    start_time = time.time()
    best_acc = 0.0
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        epoch_acc,epoch_loss = 0,0
        # разделение каждой эпохи на две фазы
        for phase in ['train','test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            total_acc, total_count = 0, 0
            interval = 4 # интервал отслеживания предсказанной точности по ходу эпохи
            for idx,(inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with tr.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss = criterion(preds, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    # параметры соответсвия
                    total_acc += (preds.argmax(1) == labels).sum().item()
                    total_count += labels.size(0)
                    if phase == 'train':
                        scheduler.step()

                    if idx % interval == 0 and idx > 0:
                        elapsed = time.time() - start_time
                        print('| epoch {:3d} | {:5d}/{:5d} batches '
                              '| accuracy {:8.3f}'.format(epoch, idx, len(dataloaders[phase]),
                                                          total_acc/total_count))

                # deep copy the model
                if phase == 'test' and total_acc > best_acc:
                    best_acc = total_acc
                    tr.save(model.state_dict(), best_model_params_path)
                if total_acc < best_acc:
                    total_acc = best_acc

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

    return model

In [ ]:
model_ft = trv.models.resnet152(weights='IMAGENET1K_V2') # загружаем предварительно обученную сеть
num_ftrs = model_ft.fc.in_features
# заменяем последний выходной слой для классификации на 3 класса
model_ft.fc = tr.nn.Linear(num_ftrs, 3)
device = tr.device('cuda')
model_ft = model_ft.to(device)

criterion = tr.nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer_ft = tr.optim.Adam(model_ft.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = tr.optim.lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:00<00:00, 264MB/s]


In [ ]:
# словари для составления расписания обучения
dataloaders = {'train':data_train,'test':data_test}
dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'test']}

In [ ]:
import time
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=12)

Epoch 0/11
----------
| epoch   0 |     4/  131 batches | accuracy    0.750
| epoch   0 |     8/  131 batches | accuracy    0.772
| epoch   0 |    12/  131 batches | accuracy    0.758
| epoch   0 |    16/  131 batches | accuracy    0.756
| epoch   0 |    20/  131 batches | accuracy    0.752
| epoch   0 |    24/  131 batches | accuracy    0.768
| epoch   0 |    28/  131 batches | accuracy    0.762
| epoch   0 |    32/  131 batches | accuracy    0.756
| epoch   0 |    36/  131 batches | accuracy    0.746
| epoch   0 |    40/  131 batches | accuracy    0.735
| epoch   0 |    44/  131 batches | accuracy    0.738
| epoch   0 |    48/  131 batches | accuracy    0.745
| epoch   0 |    52/  131 batches | accuracy    0.755
| epoch   0 |    56/  131 batches | accuracy    0.754
| epoch   0 |    60/  131 batches | accuracy    0.755
| epoch   0 |    64/  131 batches | accuracy    0.755
| epoch   0 |    68/  131 batches | accuracy    0.759
| epoch   0 |    72/  131 batches | accuracy    0.763
| epoc

In [ ]:
model_ft.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
weigh_08 = tr.load('/weights/ResNet_152_3_classes__.pt',map_location='cpu')

In [ ]:
model_ft.load_state_dict(weigh_08)

<All keys matched successfully>

In [ ]:
dataloaders = {'test':data_val}
dataset_sizes = len(dataloaders['test'])

In [ ]:
model_ = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=1)

Epoch 0/0
----------
| epoch   0 |     4/   22 batches | accuracy    0.838
| epoch   0 |     8/   22 batches | accuracy    0.889
| epoch   0 |    12/   22 batches | accuracy    0.880
| epoch   0 |    16/   22 batches | accuracy    0.890
| epoch   0 |    20/   22 batches | accuracy    0.869

Training complete in 0m 19s
Best val Acc: 13.727273


In [ ]:
def imshow(inp, title=None):
    """Display image for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.224, 0.224, 0.224])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)


def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    class_names = {0:'broken',1:'empty',2:'animal'}

    with tr.no_grad():
        for i, (inputs, labels) in enumerate(data_):
            inputs = inputs.to(device)

            outputs = model(inputs)
            _, preds = tr.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j].cpu().item()]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

visualize_model(model_ft)

### Разметка тестового датасета

In [ ]:
files = (list(paths.list_images('/train_dataset_altai/фотоловушка новое'))) # итоговый набор данных к детектированию,
len(files)

# класс для загрузки неразмеченных фото
class valid_Data(tr.utils.data.Dataset):
    def __init__(self, dirs, transform):
        self.main_dir = dirs
        self.transform = transform

    def __len__(self):
        return len(self.main_dir)

    def __getitem__(self, idx):
        img_loc = self.main_dir[idx]
        image = Image.open(img_loc).convert("RGB")
        w, h = image.size
        image = image.crop((0, 30, w, h-100))
        tensor_image = self.transform(image)
        return tensor_image

# Create the dataset
dataset = valid_Data(files,data_transforms)
check_train = tr.utils.data.DataLoader(dataset, batch_size=16, shuffle=False)

In [ ]:
next(iter(check_train))[0].shape

torch.Size([16, 3, 256, 256])

In [ ]:
from tqdm import tqdm
def pred_val(model,dataset):
    device = tr.device('cuda') if tr.cuda.is_available() else tr.device('cpu')
    model.eval()
    label = []
    for _,(inputs,path_s) in tqdm(enumerate(dataset)):
        inputs = inputs.to(device)
        with tr.no_grad():
            label.append(model(inputs))
    resul = tr.cat(label).cpu().argmax(1)
    temp = np.zeros([resul.shape[0],3])
    for i in range(len(temp)):
        temp[i,resul[i]] = 1
    return temp

label = pred_val(model_ft,check_train)

204it [09:36,  2.82s/it]
